# Inferring the goals of autonomous agents in Gen.jl

what is Gen.jl (2 sentences)

In [1]:
using Gen

## 1. Modeling an autonomous agent

Explain ( 2 sentence). Explain that a model is a probabilisttic program, and that @program is used to define a probabilistic program in Gen.jl

In [2]:
include("scene.jl")
include("path_planner.jl");

In [3]:
@program model() begin
    
    # assumed scene
    scene = Scene(0, 100, 0, 100) # the scene spans the square [0, 100] x [0, 100]
    add!(scene, Tree(Point(30, 20))) # place a tree at x=30, y=20
    add!(scene, Tree(Point(83, 80)))
    add!(scene, Tree(Point(80, 40)))
    wall_height = 30.
    add!(scene, Wall(Point(20., 40.), 1, 40., 2., wall_height))
    add!(scene, Wall(Point(60., 40.), 2, 40., 2., wall_height))
    add!(scene, Wall(Point(60.-15., 80.), 1, 15. + 2., 2., wall_height))
    add!(scene, Wall(Point(20., 80.), 1, 15., 2., wall_height))
    add!(scene, Wall(Point(20., 40.), 2, 40., 2., wall_height))    
    
    # time points at which we observe the agent's location
    observation_times = collect(linspace(0.0, 200.0, 20)) ~ "times"
    
    # assumed speed of the agent
    speed = 1.0
    
    # the starting location of the agent is a random point in the scene
    start = Point(uniform(0, 100), uniform(0, 100)) ~ "start"
    
    # the destination of the agent is a random point in the scene
    destination = Point(uniform(0, 100), uniform(0, 100)) ~ "destination"
    
    # the path of the agent from its start location to its destination
    # uses a simple 2D holonomic path planner based on RRT (path_planner.jl)
    (tree, rough_path, final_path) = plan_path(start, destination, scene)
    
    if isnull(final_path)
        
        # the agent could not find a path to its destination
        # assume it stays at the start location indefinitely
        locations = [start for _ in observation_times]
    else
        
        # the agent found a path to its destination
        # assume it moves from the start to the destinatoin along the path at constnat speed
        # sample its location along this path for each time in observation times
        locations = walk_path(get(final_path), speed, observation_times)
    end
    
    # assume that the observed locations are noisy measurements of the true locations
    # assume the noise is normally distributed with standard deviation 'noise'
    noise = 1.0
    for (i, t) in enumerate(observation_times)
        measured_x = normal(locations[i].x, noise) ~ "x$i"
        measured_y = normal(locations[i].y, noise) ~ "y$i"
    end
    
    # record other program state for rendering
    scene ~ "scene"
end;

We can run the model to generate probable scenarios:. We run a model using `@generate`, which executes a program and populates a trace with the values of the named expressions in the program.

In [4]:
trace = Trace()
@generate(trace, model())
start = value(trace, "start")
destination = value(trace, "destination")
xs = map((i) -> value(trace, "x$i"), 1:4)
ys = map((i) -> value(trace, "y$i"), 1:4)
println("start: ", start)
println("destination: ", destination)
println("xs: ", xs)
println("ys: ", ys)

start: Point(89.00966868532541,99.8767565603309)
destination: Point(18.684568462092255,35.81183806368142)
xs: [88.8667,78.0573,69.0675,57.783]
ys: [100.527,96.3618,94.7191,91.8826]


If we run it again, we get a different result:

In [5]:
trace = Trace()
@generate(trace, model())
start = value(trace, "start")
destination = value(trace, "destination")
xs = map((i) -> value(trace, "x$i"), 1:4)
ys = map((i) -> value(trace, "y$i"), 1:4)
println("start: ", start)
println("destination: ", destination)
println("xs: ", xs)
println("ys: ", ys)

start: Point(9.019889184027964,97.81749521403079)
destination: Point(52.64883111888907,5.841281459122127)
xs: [9.16418,12.9181,14.4784,14.1491]
ys: [97.4154,87.29,76.9712,68.6871]


# 2. Visualizing the probabilistic behavior of a model using a trace rendering

Printing out the values of variables is not a very good way to understand the probabilistic behavior of a program. Instead, we use a **trace rendering** to produce a visual representation of the trace. The trace rendering encodes the trace into a representation that the human visual system can quickly interpret. Here is an example trace rendering for our model that uses matplotlib:

In [6]:
javascript"""

var Gen = require("nbextensions/gen_notebook_extension/main");
var Snap = require("nbextensions/snap/snap.svg-min");

function add_wall(paper, scene_group, wall) {
    var rectangle;
    if (wall.orientation == 1) {
        // horizontal wall segment
        rectangle = paper.rect(wall.start.x, wall.start.y, wall.length, wall.thickness);
    } else if (wall.orientation == 2) {
        // vertical wall segment
        rectangle = paper.rect(wall.start.x, wall.start.y, wall.thickness, wall.length);
    }
    scene_group.add(rectangle);
}

function add_tree(paper, scene_group, tree) {
    var side = tree.size;
    var rectangle = paper.rect(tree.center.x - side/2.0, tree.center.y - side/2.0, side, side);
    rectangle.attr({fill: "#3bad29"});
    scene_group.add(rectangle);
}

function add_scene(paper, scene) {
    var group = paper.select("#scene");
    if (group) {
        group.remove();
    }
    group = paper.group();
    group.attr({id: "scene"});
    var obstacles = scene.value.obstacles;
    for (var i=0; i<obstacles.length; i++) {
        var obstacle = obstacles[i];
        switch (obstacle.name) {
            case "Wall":
                add_wall(paper, group, obstacle);
                break;
            case "Tree":
                add_tree(paper, group, obstacle);
                break;
            default:
                break;
        }
    }
}

function add_border(paper) {
    var border = paper.select("border");
    if (!border) {
        border = paper.rect(0, 0, 100, 100);
        border.attr({stroke: "#000", strokeWidth: 1, fill: "#FFF", "fill-opacity" : 0.});
    }
}

function add_start(paper, point) {
    var circle = paper.select("#start");
    if (!circle) {
        circle = paper.circle(point.value.x, point.value.y, 3);
        circle.attr({id: "start", fill: "blue", stroke: "#000",
                     "fill-opacity": 1});
    }
    circle.attr({cx: point.value.x, cy: point.value.y, 
                 strokeWidth: point.where == "interventions" ? 4 : 1});
}

function add_destination(paper, point) {
    var circle = paper.select("#destination");
    if (!circle) {
        circle = paper.circle(point.value.x, point.value.y, 3);
        circle.attr({id: "destination", fill: "red", stroke: "#000",
                     "fill-opacity": 1});
    }
    circle.attr({cx: point.value.x, cy: point.value.y, 
                 strokeWidth: start.where == "interventions" ? 4 : 1});
}

function add_path(paper, trace) {
    var times = Gen.find_choice(trace, "times");
    var num_path_points = times.value.length;
    for (var i=1; i<=num_path_points; i++) {
        var id = "path-" + i;
        var circle = paper.select("#" + id);
        var x = Gen.find_choice(trace, "x" + i);
        var y = Gen.find_choice(trace, "y" + i);
        if (!circle) {
            circle = paper.circle(x.value, y.value, 3);
            circle.attr({id: id, fill: "#EDD37F", stroke: "#000", "fill-opacity": 0.5});
        }
        circle.attr({cx: x.value, cy: y.value,
                     strokeWidth: (x.where == "interventions" || y.where == "interventions") ? 4 : 1});
    }
}

Gen.register_jupyter_renderer("agent_model_renderer", function(id, trace) {

    var paper = Snap("#" + id);
    
    // add the border
    add_border(paper);
    
    // render the scene elements
    var scene = Gen.find_choice(trace, "scene");
    if (scene) { add_scene(paper, scene); }

    // render start
    var start = Gen.find_choice(trace, "start");
    if (start) { add_start(paper, start); }

    // render destination
    var destination = Gen.find_choice(trace, "destination");
    if (destination) { add_destination(paper, destination); }

    // render points along the path
    add_path(paper, trace);
    
    // show the log weight (TODO: rename to score)
    var score = paper.select("#score");
    if (score) { score.remove(); }
    score = paper.text(50, 95, trace.log_weight.toFixed(2));
    score.attr({id: "score", "text-anchor": "middle"});
})
"""

In [7]:
renderer = JupyterInlineRenderer("agent_model_renderer")
inline(renderer)

HTML{String}("<svg id=id_YYKSOO5CUCNrejhVswPH></svg>")

Let's use this rendering to visualize a run of the our program. It will be visualized in the output of the cell above.

In [8]:
trace = Trace()
for i=1:100
    @generate(trace, model())
    render(renderer, trace)
end

Since the behavior is stochastic, we need to visualize many samples at once in a grid, to get a sense of the full distribution:

In [9]:
tiled = TiledJupyterInlineRenderer("agent_model_renderer", 3, 1, 900, 300)
inline(tiled)

HTML{String}("<svg id=id_HeIQ24ZkldvXuUKQucRR width=900 height=300 xmlns=\"http://www.w3.org/2000/svg\">\n</svg>\n<script>\n\tvar parent_svg = document.getElementById(\"id_HeIQ24ZkldvXuUKQucRR\");\n\tvar tile_width = 300.0;\n\tvar tile_height = 300.0;\n\tfor (var row=0; row < 1; row++) {\n\t\tfor (var col=0; col < 3; col++) {\n\t\t\tvar id = \"id_HeIQ24ZkldvXuUKQucRR_\" + row + \"_\" + col;\n\t\t\tvar x = tile_width * col;\n\t\t\tvar y = tile_height * row;\n\t\t\tvar svg = document.createElementNS(\"http://www.w3.org/2000/svg\", \"svg\");\n\t\t\tsvg.setAttribute(\"id\", id);\n\t\t\tsvg.setAttribute(\"x\", x);\n\t\t\tsvg.setAttribute(\"y\", y);\n\t\t\tsvg.setAttribute(\"width\", tile_width);\n\t\t\tsvg.setAttribute(\"height\", tile_height);\n\t\t\tsvg.setAttribute(\"viewBox\", \"0 0 100 100\");\n\t\t\tparent_svg.appendChild(svg);\n\t\t}\n\t}\n</script>\n")

In [10]:
traces = [begin trace = Trace(); @generate(trace, model()); trace end for i=1:3]
render(tiled, traces)

We can intervene and see what the traces look like when we fix the value of `start` and `destination` to specific values:

In [11]:
tiled = TiledJupyterInlineRenderer("agent_model_renderer", 9, 3, 900, 300)
inline(tiled)

HTML{String}("<svg id=id_tE4VoliUIF4Gw7WaRnUQ width=900 height=300 xmlns=\"http://www.w3.org/2000/svg\">\n</svg>\n<script>\n\tvar parent_svg = document.getElementById(\"id_tE4VoliUIF4Gw7WaRnUQ\");\n\tvar tile_width = 100.0;\n\tvar tile_height = 100.0;\n\tfor (var row=0; row < 3; row++) {\n\t\tfor (var col=0; col < 9; col++) {\n\t\t\tvar id = \"id_tE4VoliUIF4Gw7WaRnUQ_\" + row + \"_\" + col;\n\t\t\tvar x = tile_width * col;\n\t\t\tvar y = tile_height * row;\n\t\t\tvar svg = document.createElementNS(\"http://www.w3.org/2000/svg\", \"svg\");\n\t\t\tsvg.setAttribute(\"id\", id);\n\t\t\tsvg.setAttribute(\"x\", x);\n\t\t\tsvg.setAttribute(\"y\", y);\n\t\t\tsvg.setAttribute(\"width\", tile_width);\n\t\t\tsvg.setAttribute(\"height\", tile_height);\n\t\t\tsvg.setAttribute(\"viewBox\", \"0 0 100 100\");\n\t\t\tparent_svg.appendChild(svg);\n\t\t}\n\t}\n</script>\n")

In [12]:
trace = Trace()
intervene!(trace, "start", Point(10, 10))
intervene!(trace, "destination", Point(90, 90))
traces = Trace[]
for i=1:27
    t = deepcopy(trace)
    @generate(t, model())
    push!(traces, t)
end
render(tiled, traces)

# 3. Probabilistic inference

So far, we have simulated forward from the model, and we have intervened on some random choices and simulated the consequence. However, suppose we had observed a given sequence of locations of the agent, and we wanted to know probable goal locations? This is a query that cannot be answered simply by forward simulation of the program, because the location of the drone is a *consequence* and not a *cause* of the destination. We can easily find probable consequences given the causes, but finding probable causes given the consequences requires a bit more work.

Here is an example dataset showing measured locations for the first 15 time points:

In [13]:
points = [Point(10, 10), Point(10, 20), Point(10, 30), Point(10, 40)]

4-element Array{Point,1}:
 Point(10.0,10.0)
 Point(10.0,20.0)
 Point(10.0,30.0)
 Point(10.0,40.0)

The first step in inference is to constrain the random choices that are observed.

In [35]:
trace = Trace()
intervene!(trace, "start", Point(10, 10))
for (i, point) in enumerate(points)
    constrain!(trace, "x$i", point.x)
    constrain!(trace, "y$i", point.y)
end

Now, when we run the program in this trace, we find that the score of the trace tells us how well the trace matches the constraints.

In [15]:
tiled = TiledJupyterInlineRenderer("agent_model_renderer", 9, 3, 900, 300)
inline(tiled)

HTML{String}("<svg id=id_XOmMSoSLfPJagSu2jJpR width=900 height=300 xmlns=\"http://www.w3.org/2000/svg\">\n</svg>\n<script>\n\tvar parent_svg = document.getElementById(\"id_XOmMSoSLfPJagSu2jJpR\");\n\tvar tile_width = 100.0;\n\tvar tile_height = 100.0;\n\tfor (var row=0; row < 3; row++) {\n\t\tfor (var col=0; col < 9; col++) {\n\t\t\tvar id = \"id_XOmMSoSLfPJagSu2jJpR_\" + row + \"_\" + col;\n\t\t\tvar x = tile_width * col;\n\t\t\tvar y = tile_height * row;\n\t\t\tvar svg = document.createElementNS(\"http://www.w3.org/2000/svg\", \"svg\");\n\t\t\tsvg.setAttribute(\"id\", id);\n\t\t\tsvg.setAttribute(\"x\", x);\n\t\t\tsvg.setAttribute(\"y\", y);\n\t\t\tsvg.setAttribute(\"width\", tile_width);\n\t\t\tsvg.setAttribute(\"height\", tile_height);\n\t\t\tsvg.setAttribute(\"viewBox\", \"0 0 100 100\");\n\t\t\tparent_svg.appendChild(svg);\n\t\t}\n\t}\n</script>\n")

In [16]:
traces = Trace[]
for i=1:27
    t = deepcopy(trace)
    @generate(t, model())
    push!(traces, t)
end
render(tiled, traces)

### Importance sampling

We can use this score to filter out traces that don't match well with the observations. Specifically, we can sample a large number of traces, and pick one in proportion to the exponential of its score. We can then repreat this whole process a number of times:

In [41]:
# TODO create an HTML helper that creates an array of SVG elements, that can then be filled in one by one
# while the algorithm is running.
importance_sampling_result_renderer = TiledJupyterInlineRenderer("agent_model_renderer", 9, 3, 900, 300)
inline(importance_sampling_result_renderer)

HTML{String}("<svg id=id_vdo7k61Wn9ttQ5F0MOee width=900 height=300 xmlns=\"http://www.w3.org/2000/svg\">\n</svg>\n<script>\n\tvar parent_svg = document.getElementById(\"id_vdo7k61Wn9ttQ5F0MOee\");\n\tvar tile_width = 100.0;\n\tvar tile_height = 100.0;\n\tfor (var row=0; row < 3; row++) {\n\t\tfor (var col=0; col < 9; col++) {\n\t\t\tvar id = \"id_vdo7k61Wn9ttQ5F0MOee_\" + row + \"_\" + col;\n\t\t\tvar x = tile_width * col;\n\t\t\tvar y = tile_height * row;\n\t\t\tvar svg = document.createElementNS(\"http://www.w3.org/2000/svg\", \"svg\");\n\t\t\tsvg.setAttribute(\"id\", id);\n\t\t\tsvg.setAttribute(\"x\", x);\n\t\t\tsvg.setAttribute(\"y\", y);\n\t\t\tsvg.setAttribute(\"width\", tile_width);\n\t\t\tsvg.setAttribute(\"height\", tile_height);\n\t\t\tsvg.setAttribute(\"viewBox\", \"0 0 100 100\");\n\t\t\tparent_svg.appendChild(svg);\n\t\t}\n\t}\n</script>\n")

In [50]:
function logsumexp(arr::Vector{Float64})
    min_arr = maximum(arr)
    min_arr + log(sum(exp(arr - min_arr)))
end

approximate_samples = Trace[]
num_approximate_samples = 27
num_simulations = 10
for i=1:num_approximate_samples
    traces = Vector{Trace}(num_simulations)
    scores = Vector{Float64}(num_simulations)
    for i=1:num_simulations
        t = deepcopy(trace)
        @generate(t, model())
        scores[i] = score(t)
        traces[i] = t
    end
    weights = exp(scores - logsumexp(scores))
    weights = weights / sum(weights)
    chosen = rand(Categorical(weights))
    push!(approximate_samples, traces[chosen])
end
# TODO render incrementally!
render(importance_sampling_result_renderer, approximate_samples)

In [ ]:
# show compositing of the goal rendering...


In [17]:
# show SIR results in a tile plots

### Metropolis-Hastings Inference

In [30]:
current_trace_renderer = JupyterInlineRenderer("agent_model_renderer")
attach(current_trace_renderer, "current");
proposed_trace_renderer = JupyterInlineRenderer("agent_model_renderer")
attach(proposed_trace_renderer, "proposed");
HTML("""
        <svg id="current" height="200" viewBox="0 0 100 100"></svg>
        <svg id="proposed" height="200" viewBox="0 0 100 100"></svg>
""")

HTML{String}("        <svg id=\"current\" height=\"200\" viewBox=\"0 0 100 100\"></svg>\n        <svg id=\"proposed\" height=\"200\" viewBox=\"0 0 100 100\"></svg>\n")

In [31]:
current_trace = Trace()
intervene!(current_trace, "start", Point(10, 10))
for (i, point) in enumerate(points)
    constrain!(current_trace, "x$i", point.x)
    constrain!(current_trace, "y$i", point.y)
end
@generate(current_trace, model())
current_score = score(current_trace)
for i=1:1000
    proposed_trace = deepcopy(current_trace)
    @generate(proposed_trace, model())
    proposed_score = score(proposed_trace)
    if log(rand()) < proposed_score - current_score
        current_trace = proposed_trace
        current_score = proposed_score
    end
    render(current_trace_renderer, current_trace)
    render(proposed_trace_renderer, proposed_trace)
    sleep(0.1)
end

# 4. Improving the model

Our model above made a lot of assumptions that are unlikely to hold in the real world. For example, the agent always takes pretty direct paths from its starting location to its final destination. What if the agent is more unpredictable? What if it takes detours?

Here is a dataset that does not match our model's expectations. Let's see what happens when we try to do probabilistic inference in our model, given this data.

In [20]:
# show results (they should look bad)

This is an example of *model mis-specification*, which is when the model is not a good match for the distribution of data we are analyzing.

We can improve the model by adding the possibliltiy thathte agent takes a detour. Specifically, we add the possibility that the agent uses a waypoint and first walks from the starting locatoin to the waypoint and then from the waypoit to the final destination. Let's visualize the resulting samples:

In [21]:
# render_grid()...

Now, we can run inference as before:

In [22]:
# Do SIR with the new dataset and the improved model, and the same number of particles we used above.

In [23]:
# results for the same number of particles as above should look bad.

It looks like the results are not accurate. This is because in the new model, a random forward execution of the imporved model is a lot less likely to match the observations than a random forward exection of the original program. We can try to increase the number of samples to incrase the probability that we get one that matches the data. Note that this will take a few minutes t orun:

In [24]:
# Show results with a larger number of particles (should take < 1 min, should look okay)

This modification of the model mad einference a lot more computationally challenging, and our importance sampling algorihtm is not able to give us real-time inferneces. This motivates the need for a more sophisicated approach to probabilisitc inference.

# 5. Compiling inference with neural networks

There are a number of approaches for creating more efficient inference algorithms. We will focus on one approach, where we train a neural network to make informed guesses about he locatio nof the waypoint. First, let's understand in a bit more detail why the default inference algorihtm was slow

Suppose we knew the right waypoint:

In [25]:
trace = Trace()
constrain!(trace, "use-waypoint", true)
constrain!(trace, "waypoint", Point(0.5, 0.5))

Point(0.5,0.5)

Then the baseline importance sapmling algorithm gives reasonable inferences with fewer samples. We use this idea by training a neural network to make informed guesses about the waypoint, given the observed data as its input. We train the neural network on nany simulatoins of the program. Then, the resulting trained neural network can be used to make informed guesses about the waypooint given any observations.

In [26]:
# show the neural network, and show the training.

Let's visualize the guesses made by the neural network for a few  different datasets:

In [27]:
# show four renderings left to right of different datasets, with circles denoting the neural network's guess about the waypoint. 

Now, let's use this trained neural network to speed up inference:

In [28]:
# show the modififed SIR algorithm, using propose!

In [29]:
# show results for fewer particles, which should be noticeably faster than without the neural network.
# make an explicit comparison.